In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
from gensim.parsing.preprocessing import strip_short, remove_stopwords, preprocess_string, strip_tags, strip_punctuation
import pandas
import unidecode
from keras.utils import np_utils
import numpy as np

In [ ]:
df = pandas.read_pickle('dad_jokes.pkl')
df['joke_text_raw'] = df['title'] + " " + df['selftext']
df['joke_text_process'] = df['joke_text_raw'].apply(lambda row: unidecode.unidecode(row))
df['joke_text_process'] = df['joke_text_process'].str.lower().apply(strip_punctuation).apply(strip_tags)
df['joke_text_process'] = df['joke_text_process'].replace('[\W_]+', ' ', regex=True)

In [ ]:

df = df.drop_duplicates(subset = 'joke_text_process')
df = df[df['joke_text_process'].apply(len) > 30]

In [ ]:
chars = set(''.join((df['joke_text_process'].values.flatten())))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
df['joke_text_process'].head()


In [ ]:

X= df['joke_text_process'].values

In [ ]:
model = Sequential()
model.add(Embedding(37, 50, input_length=30))
model.add(LSTM(128))
model.add(Dense(37,activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='RMSprop', metrics = ['acc'])

model.fit(x, y, batch_size=32,
          epochs=20)

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")1
print("Saved model to disk")

In [ ]:
def joke_generate(input_seed, pred_length):
    joke_result = list(input_seed)
    print(joke_result)
    int_array = np.zeros(shape = (1,10))
    print(int_array)
    for i, ltr in enumerate(input_seed):
        int_array[0,i] = char_to_int[ltr]
    print(int_array)
    print(model.predict(int_array))
    for i in range(pred_length):
        print(i)
        new_ltr_ind = np.argmax(model.predict(int_array))
        joke_result.append(int_to_char[new_ltr_ind])
        np.roll(int_array,-1)
        int_array[0,9] = new_ltr_ind
        
        
    return ''.join(joke_result)

joke_generate("why did the chicken cross the ",20)
    